In [61]:
import random
from bisect import bisect_left


#Generate N elements for each layer and select M out of N
class Element: #define a element
 def __init__(self, eID, layer, monitored):
    self.id = eID
    self.layer = layer
    self.monitored = monitored

 def infoOut(self):
    print("Element info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", monitored = " 
          + str(self.monitored) 
         ) 

class EvtQueue:
 def __init__(self):
    self.evtID = 0 #initial number of events
    self.queue = [] #empty queue
                
 def getNextEvt(self): #obtain the head of line fault in the evt queue
    if (len(self.queue) > 0):
        currFault = self.queue[0]
        self.queue.remove(currFault) #delete the currFault 
        return currFault 
    else:
        return None
        
 def getEvtNum(self):
    return self.evtID
    
 def infoOut(self):
    for f in self.queue:
        f.infoOut()
    
 def insertEvt(self, newFault):
    def insert(seq, keys, item, keyfunc=lambda v: v):
        """Insert an item into a sorted list using a separate corresponding
       sorted keys list and a keyfunc() to extract the key from each item.
       Based on insert() method in SortedCollection recipe:
       http://code.activestate.com/recipes/577197-sortedcollection/
        """
        k = keyfunc(item)  # Get key.
        i = bisect_left(keys, k)  # Determine where to insert item.
        keys.insert(i, k)  # Insert key of item to keys list.
        seq.insert(i, item)  # Insert the item itself in the corresponding place.
    keys = [fault.evtTime for fault in self.queue]
    insert(self.queue, keys, newFault, keyfunc=lambda x: x.evtTime) #insert on the order of evtTime to the queue
    self.evtID = self.evtID + 1
    
#Generate a seqence of faults sorted in an increasing order of evtTime
class Fault: #define a fault
 def __init__(self, fID, initlayer, layer, initTime, evtTime, assocElement, impactVector):
    self.id = fID
    self.initLayer = initlayer
    self.layer = layer
    self.initTime = initTime #the inital time when the fault is generated for calculating delay
    self.evtTime = evtTime #the time when the fault is re-inserted in the event queue
    self.assocElement = assocElement #within N elements
    self.impactVector = impactVector #impact vector of this fault
    
 def infoOut(self):
    print("Fault info: id = "
          + str(self.id)
          + ", layer = " 
          + str(self.layer) 
          + ", initTime = " 
          + str(self.initTime) 
          + ", evtTime = " 
          + str(self.evtTime) 
          + ", assocElementID = " 
          + str(self.assocElement.id)
          + ", monitored = " 
          + str(self.assocElement.monitored)
          + ", impactVector = " 
          + str(self.impactVector)          
         )    
    

def init_elements():
  elements = []
  elementID = 0
  for i in range (I): #for each layer i
     j = 0
     elements_layer = []
     while j < N[i]: #generate Ni elements
        elementID = elementID + 1
        elements_layer.append(Element(elementID, i, False))
        j = j + 1
     count = 0
     assert(M[i] <= N[i]), "Total elements Ni is too small to reach Mi!"
     while count < M[i]: #select Mi elements out of Ni 
       randomElement = random.randrange(N[i])
       #print(randomElement)
       if elements_layer[randomElement].monitored == False:
          elements_layer[randomElement].monitored = True
          count = count + 1
     elements.append(elements_layer)
  return elements 
    
    
#double check    
def generate_impactVector(i): #generate an impact vector with a random impact value at layer i
    vector = [0]*I # I zeros 
    if (i in range (I)): #else assign all zeros to the ImpactVector
        vector[i] = random.random()
    return vector
                   

""" 
Initialization
"""
LASTTIME = 1000000; #simulaton running time
I = 5 # number of layers 0 ~ (I-1)
LAMDAS = [0.01, 0.08, 0.3] #the fault generation rates for each experiment for each layer
M_NS = [0.1, 0.5, 0.8]
SEED = 1
random.seed(SEED)
THRESHOLD_IMPACTVECTOR = 0.5

#Input loop variables (double check)
option = 1
D_0 = 10 # the largest delay
D_1 = 5 # a large delay

failureRatio = LAMDAS[2]
M_N = M_NS[2]
N = [100, 100, 100, 100, 100] #number of elements for layer i
M = [0, 0, 0, 0, 0] #select number of elements to monitor out of N for layer i
for i in range(len(M)):
    M[i] = N[i]*M_N
#print (M)

#Counter for monitored/detected/diagnosed faults: 
C_M = 0 #monitored 
C_D = 0 #detected
C_R = 0 #diagnosed
faultsTotal = 0 #faults != events in queue, one fault may trigger multiple events at different layers for option 4
delayTotal = 0
FaultRecoveryDelay = 0

elements = init_elements() #initialize elements monitored and not-monitored at each layer
queue = EvtQueue()
# for each layer, insert the initial fault with evtTime = 0 in the queue


for i in range(I): 
    evtTime = 0
    elements_layer = elements[i] #find corresponding elements_layer 
    elementIndex = random.randint(0, len(elements_layer)-1) #randomly (uniformaly) assign fault to Ni elements at layer i
    newFault = Fault(faultsTotal, i, i, evtTime, evtTime, elements_layer[elementIndex], generate_impactVector(i+1)) #Topdown to generator impact vector to the lower i + 1 layer
    queue.insertEvt(newFault)
    faultsTotal = faultsTotal + 1
#queue.infoOut()


""" 
Fault generation
"""
#start to process a Head of Line fault in the event queue
currFault = queue.getNextEvt()
while (currFault != None):
    #currFault.infoOut()
    
    #insert the next new fault based on the info of the currFault
    evtTime = currFault.evtTime + random.expovariate(failureRatio)
    if evtTime <= LASTTIME:
        elementIndex = random.randint(0, len(elements[currFault.layer])-1) #generate a new fault with the same layer as the current fault
        newFault = Fault(faultsTotal, currFault.layer, currFault.layer, evtTime, evtTime, 
                         elements[currFault.layer][elementIndex], generate_impactVector(i+1))     
        queue.insertEvt(newFault)
        faultsTotal = faultsTotal + 1
        #newFault.infoOut()
        
    """ 
    Monitoring
    """        
    #add code here to process the currFault
    #Calculate counters
    if currFault.assocElement.monitored == True:
         delayTotal = delayTotal + D_1
         C_M = C_M + 1
    else:
         delayTotal = delayTotal + D_0
          
    """ 
    Fault generation
    """
    currFault = queue.getNextEvt()
    

print("Total num of faults: " + str(faultsTotal) + ", total events = " + str(queue.getEvtNum()))
print("C_M = " + str(C_M) + ", The ratio of monitored faults = " + str(C_M/faultsTotal))
print("C_D = " + str(C_D))
print("C_R = " + str(C_R))
print("FailureRate = " + str(failureRatio) + ", M/N = " + str(M_N) + ", FaultRecoverDelay = " + str(delayTotal/faultsTotal))







Total num of faults: 1500428, total events = 1500428
C_M = 1200432, The ratio of monitored faults = 0.8000597162942841
C_D = 0
C_R = 0
FailureRate = 0.3, M/N = 0.8, FaultRecoverDelay = 5.99970141852858
